In [1]:
from __future__ import print_function
import collections
from flask import jsonify
import requests
import os
import re
import json, codecs
import tensorflow as tf
from keras.models import Sequential, load_model
from keras.layers import Dense, Activation, Embedding, Flatten, Dropout, SpatialDropout1D, TimeDistributed, Reshape, Lambda
from keras.layers import LSTM
from keras.optimizers import RMSprop, Adam, SGD
from keras import backend as K
from keras.utils import to_categorical, np_utils
from keras.callbacks import ModelCheckpoint
from keras.models import model_from_json, Model
import keras.backend as K
from keras.layers import LSTM, Input
from sklearn.model_selection import train_test_split
import numpy as np
import argparse
import pdb
from gensim.models import KeyedVectors
import pandas as pd
from nltk.corpus import stopwords
from nltk.corpus import wordnet
from nltk.stem.wordnet import WordNetLemmatizer
import string
from sklearn.preprocessing import LabelEncoder
from matplotlib import pyplot
from sklearn.externals import joblib

C:\Users\Nicholas\AppData\Local\Programs\Python\Python36\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.
C:\Users\Nicholas\AppData\Local\Programs\Python\Python36\lib\site-packages\gensim\utils.py:1209: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [3]:
en_model = KeyedVectors.load_word2vec_format('data/wiki-news-300d-1M.vec')

In [61]:
four_emotions=load_model('models/four_emotions.h5')

In [27]:
negative = ['not', 'no', 'neither', 'nor', 'but', 'however', 'although', 'nonetheless', 'despite', 'except',
                         'even though', 'yet']
stop = list(set(stopwords.words('english')))
for neg in negative:
    for stopword in stop:
        if stopword==neg:
            stop.remove(stopword)
exclude = set(string.punctuation)
exclude.add('\n')
encoder=LabelEncoder()

In [45]:
def clean(doc):
    lemma=WordNetLemmatizer()
    stop_free = " ".join([i for i in doc.lower().split() if i not in stop])
    punc_free = "".join([ch for ch in stop_free if ch not in exclude])
    re.sub(r'\n', '', punc_free)
    normalized = " ".join([lemma.lemmatize(word) for word in punc_free.split()])
    return normalized

In [46]:
def vec_word(li):
    total_vecs=[]
    for word in li:
        if word in en_model.vocab:
            vector = en_model[word]
            total_vecs.append(vector)
    return np.array(total_vecs)

In [47]:
def word_splits(series):
    word_splits=series.str.split(' ')
    return word_splits

<h3>ISEAR</h3>

In [ ]:
isear=pd.read_csv('isear_1.csv', header=-1)
isear=isear.drop(2 , axis=1)
isear[0][isear[0]=='guit']='guilt'

In [38]:
X=isear[1]
X=X.apply(clean)
y=isear[0]

In [97]:
splits=word_splits(X)
numbers_series=splits.apply(vec_word)

In [113]:
def transform_input(text):
    X=pd.Series(text).apply(clean)
    splits=word_splits(X)
    numbers_series=splits.apply(vec_word)
    num_words=len(numbers_series)
    X_1 = []
    for index in range(0, num_words):
        doc=numbers_series.iloc[index]
        for i in range(3, len(doc)):
            X_1.append(doc[i-3:i])
    return np.array(X_1), len(X_1)

In [59]:
def transform_textform(text):
    X=pd.Series(text).apply(clean)
    splits=word_splits(X)
    numbers_series=splits
    X_1 = []
    for index in range(0, len(numbers_series)):
        doc=numbers_series.iloc[index]
        for i in range(3, len(doc)):
            X_1.append(doc[i-3:i])
    return X_1

In [408]:
def transform_y(y):
    encoder.fit(y)
    y=encoder.transform(y)
    y_1=np_utils.to_categorical(y)
    return y_1

In [99]:
X_1 = []
y_1 = []
for index in range(0, len(numbers_series)):
    doc=numbers_series.iloc[index]
    for i in range(3, len(doc)):
        X_1.append(doc[i-3:i])
        y_1.append(y.iloc[index])

In [104]:
X_2=np.array(X_1)

In [105]:
X_2.shape

(53460, 3, 300)

In [56]:
encoder.fit(y_1)
y_1=encoder.transform(y_1)
y=np_utils.to_categorical(y_1)
y

array([[0., 0., 0., ..., 1., 0., 0.],
       [0., 0., 0., ..., 1., 0., 0.],
       [0., 0., 0., ..., 1., 0., 0.],
       ...,
       [0., 0., 1., ..., 0., 0., 0.],
       [0., 0., 1., ..., 0., 0., 0.],
       [0., 0., 1., ..., 0., 0., 0.]], dtype=float32)

In [231]:
X_train, X_test, y_train, y_test = train_test_split(X_1, y, test_size = 0.2, random_state = 0)
X_train, y_train = np.array(X_train), np.array(y_train)

In [234]:
X_train=np.array(X_train)
X_test=np.array(X_test)
y_train=np.array(y_train)
y_test=np.array(y_test)

In [235]:
y_test.shape

(10692, 7)

In [236]:
y_train.shape

(42768, 7)

In [237]:
len(X_train)

42768

In [110]:
X_train, y_train = np.array(X_train), np.array(y_train)

In [111]:
X_train.shape

(53460, 3, 300)

In [112]:
y_train.shape

(53460,)

In [128]:
# Initialising the RNN
regressor = Sequential()

# Adding the first LSTM layer and some Dropout regularisation
regressor.add(LSTM(units = 50, return_sequences = True, input_dim=300))
regressor.add(Dropout(0.2))

# Adding a second LSTM layer and some Dropout regularisation
regressor.add(LSTM(units = 50, return_sequences = True))
regressor.add(Dropout(0.2))

# Adding a third LSTM layer and some Dropout regularisation
regressor.add(LSTM(units = 50, return_sequences = True))
regressor.add(Dropout(0.2))

# Adding a fourth LSTM layer and some Dropout regularisation
regressor.add(LSTM(units = 50))
regressor.add(Dropout(0.2))

# Adding the output layer
regressor.add(Dense(units = 7))

C:\Users\Nicholas\AppData\Local\Programs\Python\Python36\lib\site-packages\ipykernel_launcher.py:5: UserWarning: The `input_dim` and `input_length` arguments in recurrent layers are deprecated. Use `input_shape` instead.
  """
C:\Users\Nicholas\AppData\Local\Programs\Python\Python36\lib\site-packages\ipykernel_launcher.py:5: UserWarning: Update your `LSTM` call to the Keras 2 API: `LSTM(units=50, return_sequences=True, input_shape=(None, 300...)`
  """


In [141]:
# Compiling the RNN
regressor.compile(optimizer = 'adam', metrics=['accuracy'], loss = 'mean_squared_error')
regressor.fit(X_train, y_train, epochs = 25, batch_size = 32)

Epoch 1/25
42768/42768 [==============================] - 18s 419us/step - loss: 0.0495 - acc: 0.7586
Epoch 2/25
42768/42768 [==============================] - 15s 358us/step - loss: 0.0490 - acc: 0.7635
Epoch 3/25
42768/42768 [==============================] - 15s 358us/step - loss: 0.0488 - acc: 0.7621
Epoch 4/25
42768/42768 [==============================] - 15s 359us/step - loss: 0.0488 - acc: 0.7621
Epoch 5/25
42768/42768 [==============================] - 15s 359us/step - loss: 0.0486 - acc: 0.7650
Epoch 6/25
42768/42768 [==============================] - 15s 360us/step - loss: 0.0487 - acc: 0.7630
Epoch 7/25
42768/42768 [==============================] - 15s 362us/step - loss: 0.0485 - acc: 0.7640
Epoch 8/25
42768/42768 [==============================] - 16s 370us/step - loss: 0.0481 - acc: 0.7680
Epoch 9/25
42768/42768 [==============================] - 16s 367us/step - loss: 0.0481 - acc: 0.7662
Epoch 10/25
42768/42768 [==============================] - 16s 384us/step - loss: 

In [142]:
regressor.evaluate(X_test, y_test)

10692/10692 [==============================] - 1s 117us/step


[0.10446360716760003, 0.48110736999625886]

In [135]:
regressor.metrics_names

['loss']

<h3>Initial model with .48 test accuracy</h3>

<h3>Saving and loading</h3>

In [535]:
model_json = regressor.to_json()
with open("emotions_model_6.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
regressor.save_weights("emotions_model_6.h5")
print("Saved model to disk")

Saved model to disk


In [4]:
# load json and create model
json_file = open('emotions_regressor.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)
# load weights into new model
loaded_model.load_weights("emotions_regressor.h5")
print("Loaded model from disk")

Loaded model from disk


In [58]:
try:
    print(loaded_model.predict(transform_input('I like pie a whole bunch a lot')))
except ValueError:
    print('Input size too small')

[[0.09755597 0.04315098 0.17637494 0.07367538 0.43375415 0.07726857
  0.09988091]
 [0.56483126 0.07285608 0.13253172 0.04942662 0.09893434 0.02065588
  0.0610196 ]]


In [246]:
def predict_avg(text):
    X, X_len=transform_input(text)
    prediction=loaded_model.predict(transform_input(text)[0])
    prediction=np.mean(prediction, axis=0)
    return prediction

In [153]:
pie_mat=predict_avg('I hate all aliens they are evil bastards')
pie_mat

array([0.02197286, 0.90678847, 0.01035543, 0.01425846, 0.00490067,
       0.01707822, 0.01889859], dtype=float32)

0:anger

1:disgust

2:fear

3:guilt

4: joy

5:sadness

6:shame

loaded_model (model 1) doesn't do very well. Let's try to make another

<h3>Model with unknown timespan</h3>

In [203]:
X=isear[1]
X=X.apply(clean)
y=isear[0]

In [204]:
splits=word_splits(X)
numbers_series=splits.apply(vec_word)

In [185]:
def transform_no_window(text):
    X=pd.Series(text).apply(clean)
    splits=word_splits(X)
    numbers_series=splits.apply(vec_word)
    return numbers_series

In [187]:
X=X.apply(transform_no_window)

In [196]:
X_1=X

In [197]:
X_1=X_1[0]

0       [[0.032, 0.0381, -0.0299, -0.0745, -0.0624, -0...
1       [[-0.0036, -0.1675, 0.0635, -0.0249, 0.0098, -...
2       [[0.079, -0.0397, 0.005, 0.0136, -0.0226, -0.0...
3       [[-0.0432, 0.037, -0.0039, 0.0524, -0.1216, -0...
4       [[0.0586, -0.037, -0.081, -0.1985, -0.1255, -0...
5       [[0.1129, -0.0975, 0.0769, -0.071, 0.0645, -0....
6       [[-0.0372, 0.0615, 0.0167, -0.0382, 0.0207, -0...
7       [[0.1912, -0.0745, 0.0554, -0.0277, -0.0276, 0...
8       [[0.0092, -0.0123, -0.0336, 0.0269, 0.031, 0.0...
9       [[-0.016, -0.0003, -0.1684, 0.0899, -0.02, -0....
10      [[0.0103, 0.0984, 0.1326, 0.0845, 0.0898, -0.1...
11      [[-0.0368, -0.0336, 0.0182, 0.0258, -0.0327, -...
12      [[-0.1362, 0.1148, -0.0027, -0.0033, 0.1185, -...
13      [[-0.0432, 0.037, -0.0039, 0.0524, -0.1216, -0...
14      [[-0.0079, -0.0009, -0.0813, -0.2044, 0.1367, ...
15      [[0.0458, -0.0362, 0.0156, 0.0027, -0.053, 0.0...
16      [[0.0458, -0.0362, 0.0156, 0.0027, -0.053, 0.0...
17      [[0.04

In [201]:
X_2=np.array(X_1)
X_2.shape

(7516,)

In [202]:
X_2[0:3]

array([array([[ 0.032 ,  0.0381, -0.0299, ...,  0.1316,  0.1363,  0.044 ],
       [-0.0372,  0.0615,  0.0167, ...,  0.1405, -0.0883, -0.1164],
       [ 0.0622,  0.0628,  0.2393, ...,  0.1802, -0.0394, -0.0243],
       ...,
       [ 0.0104,  0.1101, -0.0982, ...,  0.1598, -0.0329,  0.0798],
       [ 0.0055, -0.1195,  0.0729, ...,  0.2087, -0.0779, -0.0624],
       [-0.1902, -0.1442, -0.1249, ...,  0.2267, -0.0037,  0.0915]],
      dtype=float32),
       array([[-0.0036, -0.1675,  0.0635, ...,  0.1275,  0.1066, -0.0399],
       [ 0.0863, -0.0405,  0.0437, ...,  0.1703, -0.3191, -0.0287],
       [-0.1006, -0.0988, -0.1582, ...,  0.2289,  0.1009, -0.018 ],
       ...,
       [ 0.1084,  0.1093, -0.1632, ..., -0.0037, -0.2236, -0.2337],
       [-0.0112, -0.1144, -0.0171, ...,  0.1761,  0.0193, -0.0004],
       [ 0.1013,  0.0365,  0.0062, ...,  0.0657,  0.0512, -0.0055]],
      dtype=float32),
       array([[ 0.079 , -0.0397,  0.005 , ...,  0.1426, -0.1237, -0.0863],
       [ 0.1821, -0.0348,

In [163]:
encoder=LabelEncoder()
encoder.fit(y)
y=encoder.transform(y)
y=np_utils.to_categorical(y)
y

array([[0., 0., 0., ..., 1., 0., 0.],
       [0., 0., 1., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 1., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 1., ..., 0., 0., 0.]], dtype=float32)

In [164]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)
X_train, y_train = np.array(X_train), np.array(y_train)

In [171]:
X_train[1].shape

(16, 300)

Input shape

3D tensor with shape (batch_size, timesteps, input_dim).

In [240]:
# Initialising the RNN
regressor = Sequential()

# Adding the first LSTM layer and some Dropout regularisation
regressor.add(LSTM(units = 50, return_sequences = True, input_dim=300))
regressor.add(Dropout(0.2))

# Adding a second LSTM layer and some Dropout regularisation
regressor.add(LSTM(units = 50, return_sequences = True))
regressor.add(Dropout(0.2))

# Adding a third LSTM layer and some Dropout regularisation
regressor.add(LSTM(units = 50, return_sequences = True))
regressor.add(Dropout(0.2))

# Adding a fourth LSTM layer and some Dropout regularisation
regressor.add(LSTM(units = 50))
regressor.add(Dropout(0.2))

# Adding the output layer
regressor.add(Dense(units = 7))

C:\Users\Nicholas\AppData\Local\Programs\Python\Python36\lib\site-packages\ipykernel_launcher.py:5: UserWarning: The `input_dim` and `input_length` arguments in recurrent layers are deprecated. Use `input_shape` instead.
  """
C:\Users\Nicholas\AppData\Local\Programs\Python\Python36\lib\site-packages\ipykernel_launcher.py:5: UserWarning: Update your `LSTM` call to the Keras 2 API: `LSTM(units=50, return_sequences=True, input_shape=(None, 300...)`
  """


In [241]:
# Compiling the RNN
regressor.compile(optimizer = 'adam', metrics=['accuracy'], loss = 'mean_squared_error')

In [ ]:
regressor.fit(new_input, y_train, epochs = 25, batch_size = 32)

In [250]:
I = Input(shape=(None, 300)) # unknown timespan, fixed feature size
lstm = LSTM(20)
dense=Dense(units = 7)
dense_out=dense(I)
f = K.function(inputs=[I], outputs=[lstm(I)])

In [257]:
new_input=list(new_input)

In [259]:
type(new_input)

list

In [ ]:
model=Model(input=[I], output=[dense_out])
model.compile(optimizer = 'adam', metrics=['accuracy'], loss = 'mean_squared_error')
model.fit(new_input)

In [216]:
X_train[0].shape 

(21, 300)

In [231]:
new_input=list(X_train)
new_input=[np.reshape(arr, (1, arr.shape[0], 300)) for arr in new_input]

In [232]:
f(new_input)

[array([[-0.00189861, -0.02905425,  0.01028993,  0.05693588,  0.12599891,
         -0.07233123, -0.15039034, -0.03753458,  0.0746619 ,  0.03368369,
         -0.01838236, -0.05823456, -0.0047072 , -0.00749667, -0.03643454,
          0.03162642,  0.02647312,  0.02489861,  0.06106438,  0.00751286]],
       dtype=float32)]

In [ ]:
#window
X_1 = []
    for index in range(0, len(numbers_series)):
        doc=numbers_series.iloc[index]
        for i in range(3, len(doc)):
            X_1.append(doc[i-3:i])
    return np.array(X_1)

<h3>Second model</h3>

In [165]:
# Initialising the RNN
regressor = Sequential()

# Adding the first LSTM layer and some Dropout regularisation
regressor.add(LSTM(units = 100, return_sequences = True, input_dim=300))
regressor.add(Dropout(0.2))

# Adding a second LSTM layer and some Dropout regularisation
regressor.add(LSTM(units = 100, return_sequences = True))
regressor.add(Dropout(0.2))

# Adding a third LSTM layer and some Dropout regularisation
regressor.add(LSTM(units = 100, return_sequences = True))
regressor.add(Dropout(0.2))

# Adding a LSTM layer and some Dropout regularisation
regressor.add(LSTM(units = 100, return_sequences = True))
regressor.add(Dropout(0.2))

# Adding a LSTM layer and some Dropout regularisation
regressor.add(LSTM(units = 100, return_sequences = True))
regressor.add(Dropout(0.2))

# Adding a LSTM layer and some Dropout regularisation
regressor.add(LSTM(units = 100))
regressor.add(Dropout(0.2))

# Adding the output layer
regressor.add(Dense(units = 7, kernel_initializer = 'uniform', activation = 'softmax'))

C:\Users\Nicholas\AppData\Local\Programs\Python\Python36\lib\site-packages\ipykernel_launcher.py:5: UserWarning: The `input_dim` and `input_length` arguments in recurrent layers are deprecated. Use `input_shape` instead.
  """
C:\Users\Nicholas\AppData\Local\Programs\Python\Python36\lib\site-packages\ipykernel_launcher.py:5: UserWarning: Update your `LSTM` call to the Keras 2 API: `LSTM(units=100, return_sequences=True, input_shape=(None, 300...)`
  """


In [161]:
adam=Adam(lr=0.0005, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False)

In [166]:
# Compiling the RNN
regressor.compile(optimizer = adam, metrics=['accuracy'], loss = 'categorical_crossentropy')

In [173]:
regressor.fit(X_train, y_train, epochs = 25, batch_size = 32)

Epoch 1/25
42768/42768 [==============================] - 32s 748us/step - loss: 0.8835 - acc: 0.6842
Epoch 2/25
42768/42768 [==============================] - 32s 737us/step - loss: 0.8675 - acc: 0.6885
Epoch 3/25
42768/42768 [==============================] - 32s 743us/step - loss: 0.8545 - acc: 0.6947
Epoch 4/25
42768/42768 [==============================] - 31s 735us/step - loss: 0.8418 - acc: 0.7000
Epoch 5/25
42768/42768 [==============================] - 31s 732us/step - loss: 0.8274 - acc: 0.7037
Epoch 6/25
42768/42768 [==============================] - 31s 732us/step - loss: 0.8193 - acc: 0.7093
Epoch 7/25
42768/42768 [==============================] - 32s 750us/step - loss: 0.8059 - acc: 0.71211s - l
Epoch 8/25
42768/42768 [==============================] - 32s 747us/step - loss: 0.8002 - acc: 0.7122
Epoch 9/25
42768/42768 [==============================] - 32s 757us/step - loss: 0.7883 - acc: 0.7179
Epoch 10/25
42768/42768 [==============================] - 33s 765us/step - 

In [174]:
regressor.evaluate(X_test, y_test)

10692/10692 [==============================] - 2s 215us/step


[1.791913282456489, 0.48054620276842497]

In [175]:
model_json = regressor.to_json()
with open("emotions_regressor_2.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
regressor.save_weights("emotions_regressor_2.h5")
print("Saved model to disk")

Saved model to disk


<h3>Third model</h3>

In [177]:
# Initialising the RNN
regressor = Sequential()

# Adding the first LSTM layer and some Dropout regularisation
regressor.add(LSTM(units = 100, return_sequences = True, input_dim=300))
regressor.add(Dropout(0.5))

# Adding a second LSTM layer and some Dropout regularisation
regressor.add(LSTM(units = 100, return_sequences = True))
regressor.add(Dropout(0.5))

# Adding a third LSTM layer and some Dropout regularisation
regressor.add(LSTM(units = 100, return_sequences = True))
regressor.add(Dropout(0.5))

# Adding a LSTM layer and some Dropout regularisation
regressor.add(LSTM(units = 100, return_sequences = True))
regressor.add(Dropout(0.5))

# Adding a LSTM layer and some Dropout regularisation
regressor.add(LSTM(units = 100, return_sequences = True))
regressor.add(Dropout(0.5))

# Adding a LSTM layer and some Dropout regularisation
regressor.add(LSTM(units = 100))
regressor.add(Dropout(0.5))

# Adding the output layer
regressor.add(Dense(units = 7, kernel_initializer = 'uniform', activation = 'softmax'))

C:\Users\Nicholas\AppData\Local\Programs\Python\Python36\lib\site-packages\ipykernel_launcher.py:5: UserWarning: The `input_dim` and `input_length` arguments in recurrent layers are deprecated. Use `input_shape` instead.
  """
C:\Users\Nicholas\AppData\Local\Programs\Python\Python36\lib\site-packages\ipykernel_launcher.py:5: UserWarning: Update your `LSTM` call to the Keras 2 API: `LSTM(units=100, return_sequences=True, input_shape=(None, 300...)`
  """


In [180]:
adam=Adam(lr=0.0007, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False)

In [181]:
# Compiling the RNN
regressor.compile(optimizer = adam, metrics=['accuracy'], loss = 'categorical_crossentropy')

In [196]:
hist=regressor.fit(X_train, y_train, epochs = 100, batch_size = 32, validation_data=(X_test, y_test), shuffle=False)

Train on 42768 samples, validate on 10692 samples
Epoch 1/100
42768/42768 [==============================] - 33s 783us/step - loss: 0.5579 - acc: 0.8134 - val_loss: 2.3434 - val_acc: 0.4560
Epoch 2/100
 3776/42768 [=>............................] - ETA: 28s - loss: 0.5140 - acc: 0.8284

KeyboardInterrupt: 

In [189]:
regressor.evaluate(X_test, y_test)

10692/10692 [==============================] - 2s 215us/step


[2.214059761745226, 0.4675458286569398]

In [ ]:
pyplot.plot(hist.history['loss'][500:])
pyplot.plot(hist.history['val_loss'][500:])
pyplot.title('model train vs validation loss')
pyplot.ylabel('loss')
pyplot.xlabel('epoch')
pyplot.legend(['train', 'validation'], loc='upper right')
pyplot.show()

<h3>Fourth model</h3>

In [226]:
# Initialising the RNN
regressor = Sequential()

# Adding the first LSTM layer and some Dropout regularisation
regressor.add(LSTM(units = 100, return_sequences = True, input_dim=300))
regressor.add(SpatialDropout1D(0.5))

# Adding a second LSTM layer and some Dropout regularisation
regressor.add(LSTM(units = 100, return_sequences = True))
regressor.add(SpatialDropout1D(0.5))

# Adding a third LSTM layer and some Dropout regularisation
regressor.add(LSTM(units = 100, return_sequences = True))
regressor.add(SpatialDropout1D(0.5))

# Adding a LSTM layer and some Dropout regularisation
regressor.add(LSTM(units = 100, return_sequences = True))
regressor.add(SpatialDropout1D(0.5))

# Adding a LSTM layer and some Dropout regularisation
regressor.add(LSTM(units = 100, return_sequences = True))
regressor.add(SpatialDropout1D(0.5))

# Adding a LSTM layer
regressor.add(LSTM(units = 100))

# Adding the output layer
regressor.add(Dense(units = 7, kernel_initializer = 'uniform', activation = 'softmax'))

C:\Users\Nicholas\AppData\Local\Programs\Python\Python36\lib\site-packages\ipykernel_launcher.py:5: UserWarning: The `input_dim` and `input_length` arguments in recurrent layers are deprecated. Use `input_shape` instead.
  """
C:\Users\Nicholas\AppData\Local\Programs\Python\Python36\lib\site-packages\ipykernel_launcher.py:5: UserWarning: Update your `LSTM` call to the Keras 2 API: `LSTM(units=100, return_sequences=True, input_shape=(None, 300...)`
  """


In [227]:
# Compiling the RNN
regressor.compile(optimizer = 'adam', metrics=['accuracy'], loss = 'categorical_crossentropy')

In [ ]:
X_train.shape

In [238]:
model_4_hist=regressor.fit(X_train, y_train, epochs = 100, batch_size = 32, validation_data=(X_test, y_test), shuffle=False)

Train on 42768 samples, validate on 10692 samples
Epoch 1/100
42768/42768 [==============================] - 43s 1000us/step - loss: 1.8514 - acc: 0.2293 - val_loss: 1.7700 - val_acc: 0.2788
Epoch 2/100
42768/42768 [==============================] - 34s 789us/step - loss: 1.7607 - acc: 0.2812 - val_loss: 1.7198 - val_acc: 0.3098
Epoch 3/100
42768/42768 [==============================] - 34s 789us/step - loss: 1.7203 - acc: 0.3059 - val_loss: 1.6828 - val_acc: 0.3389
Epoch 4/100
42768/42768 [==============================] - 34s 791us/step - loss: 1.6804 - acc: 0.3327 - val_loss: 1.6587 - val_acc: 0.3534
Epoch 5/100
42768/42768 [==============================] - 34s 795us/step - loss: 1.6542 - acc: 0.3474 - val_loss: 1.6329 - val_acc: 0.3690
Epoch 6/100
42768/42768 [==============================] - 34s 789us/step - loss: 1.6295 - acc: 0.3625 - val_loss: 1.6260 - val_acc: 0.3715
Epoch 7/100
42768/42768 [==============================] - 34s 792us/step - loss: 1.6074 - acc: 0.3700 - val_

<h3>Model 5: Stateful learning on size 1 elements</h3>

In [ ]:
X=isear[1]
X=X.apply(clean)
y=isear[0]

In [269]:
def transform_train(text):
    X=pd.Series(text).apply(clean)
    splits=word_splits(X)
    numbers_series=splits.apply(vec_word)
    num_words=len(numbers_series)
    X_1 = []
    y_1=  []
    for index in range(0, num_words):
        doc=numbers_series.iloc[index]
        for i in range(len(doc)):
            X_1.append(doc[i])
            y_1.append(y.iloc[index])
    return X_1, y_1

In [290]:
X_1, y_1=transform_train(X)

In [291]:
encoder=LabelEncoder()
encoder.fit(y_1)
y_1=encoder.transform(y_1)
y_1=np_utils.to_categorical(y_1)
y_1

array([[0., 0., 0., ..., 1., 0., 0.],
       [0., 0., 0., ..., 1., 0., 0.],
       [0., 0., 0., ..., 1., 0., 0.],
       ...,
       [0., 0., 1., ..., 0., 0., 0.],
       [0., 0., 1., ..., 0., 0., 0.],
       [0., 0., 1., ..., 0., 0., 0.]], dtype=float32)

In [293]:
X_1=np.array(X_1)

In [294]:
X_1=np.reshape(X_1, (X_1.shape[0], 1, X_1.shape[1]))

In [296]:
X_train, X_test, y_train, y_test = train_test_split(X_1, y_1, test_size = 0.2, random_state = 0)
X_train, y_train = np.array(X_train), np.array(y_train)

In [312]:
X_train.shape

(58205, 1, 300)

In [322]:
# Initialising the RNN
regressor = Sequential()

# Adding the first LSTM layer and some Dropout regularisation
regressor.add(LSTM(units = 25, return_sequences = True, stateful=True,  batch_input_shape=(35, 1, 300)))
regressor.add(Dropout(0.2))

# Adding a second LSTM layer and some Dropout regularisation
regressor.add(LSTM(units = 25, return_sequences = True))
regressor.add(Dropout(0.2))

# Adding a third LSTM layer and some Dropout regularisation
regressor.add(LSTM(units = 25, return_sequences = True))
regressor.add(Dropout(0.2))

# Adding a fourth LSTM layer and some Dropout regularisation
regressor.add(LSTM(units = 25))
regressor.add(Dropout(0.2))

# Adding the output layer
regressor.add(Dense(units = 7))

In [329]:
# Compiling the RNN
regressor.compile(optimizer = 'adam', metrics=['accuracy'], loss = 'mean_squared_error')
#Fitting to training set
regressor.fit(X_train, y_train, epochs = 25, batch_size = 35)

Epoch 1/25
58205/58205 [==============================] - 11s 187us/step - loss: 0.1149 - acc: 0.2824
Epoch 2/25
58205/58205 [==============================] - 8s 140us/step - loss: 0.1150 - acc: 0.2831
Epoch 3/25
58205/58205 [==============================] - 8s 141us/step - loss: 0.1152 - acc: 0.2803
Epoch 4/25
58205/58205 [==============================] - 8s 140us/step - loss: 0.1150 - acc: 0.2807
Epoch 5/25
58205/58205 [==============================] - 8s 141us/step - loss: 0.1151 - acc: 0.2816
Epoch 6/25
58205/58205 [==============================] - 8s 141us/step - loss: 0.1151 - acc: 0.2788
Epoch 7/25
58205/58205 [==============================] - 8s 142us/step - loss: 0.1153 - acc: 0.2792
Epoch 8/25
58205/58205 [==============================] - 8s 141us/step - loss: 0.1153 - acc: 0.2784
Epoch 9/25
58205/58205 [==============================] - 8s 142us/step - loss: 0.1153 - acc: 0.2774
Epoch 10/25
58205/58205 [==============================] - 8s 143us/step - loss: 0.1154 - 

<h3> Only run below once </h3>

In [325]:
X_test, y_test= X_test[:13475], y_test[:13475]

In [330]:
regressor.evaluate(X_test, y_test, batch_size=35)

13475/13475 [==============================] - 2s 113us/step


[0.11635366178952254, 0.2500185603106564]

In [328]:
regressor.metrics_names

['loss', 'acc']

<i>Notes on size 1: Not as good as size 3

<h3>Model 6: timestep size 6 (double of first model)</h3>

In [331]:
X=isear[1]
X=X.apply(clean)
y=isear[0]

In [85]:
def transform_t6_train(text):
    X=pd.Series(text).apply(clean)
    splits=word_splits(X)
    numbers_series=splits.apply(vec_word)
    num_docs=len(numbers_series)
    X_1 = []
    y_1= []
    for index in range(0, num_docs):
        doc=numbers_series.iloc[index]
        for i in range(6, len(doc)):
            X_1.append(doc[i-6:i])
            y_1.append(y.iloc[index])
    return X_1, y_1

In [63]:
def transform_t6_input(text):
    X=pd.Series(text).apply(clean)
    splits=word_splits(X)
    numbers_series=splits.apply(vec_word)
    num_docs=len(numbers_series)
    if numbers_series[0].size!=0:
        for index, doc in enumerate(numbers_series):
            while len(doc)<7:
                orig_doc=doc.copy()
                orig_doc=list(orig_doc)
                doc=list(doc)
                for word in orig_doc:
                    doc.append(word)
                modified=True
            numbers_series.iloc[index]=np.array(doc)
        X_1 = []
        if num_docs>1:
            for index in range(0, num_docs):
                doc=numbers_series.iloc[index]
                for i in range(6, len(doc)):
                    X_1.append(doc[i-6:i])
        else:
            doc=numbers_series.iloc[0]
            for i in range(6, len(doc)):
                    X_1.append(doc[i-6:i])
        return np.array(X_1)
    else:
        return 'not_found'

In [366]:
pie=transform_t6_input('I like to eat pie a lot it is yummy and very delicious')

In [333]:
X_1, y_1=transform_t6_train(X)

In [336]:
X_1=np.array(X_1)
X_1.shape

(32468, 6, 300)

In [335]:
encoder=LabelEncoder()
encoder.fit(y_1)
y_1=encoder.transform(y_1)
y_1=np_utils.to_categorical(y_1)
y_1

array([[0., 0., 0., ..., 1., 0., 0.],
       [0., 0., 0., ..., 1., 0., 0.],
       [0., 0., 0., ..., 1., 0., 0.],
       ...,
       [0., 0., 1., ..., 0., 0., 0.],
       [0., 0., 1., ..., 0., 0., 0.],
       [0., 0., 1., ..., 0., 0., 0.]], dtype=float32)

In [337]:
X_train, X_test, y_train, y_test = train_test_split(X_1, y_1, test_size = 0.2, random_state = 0)
X_train, y_train = np.array(X_train), np.array(y_train)

In [1]:
def create_lstm(num_emotions):
    # Initialising the RNN
    regressor = Sequential()

    # Adding the first LSTM layer and some Dropout regularisation
    regressor.add(LSTM(units = 25, return_sequences = True, input_dim=300))
    regressor.add(Dropout(0.2))

    # Adding a second LSTM layer and some Dropout regularisation
    regressor.add(LSTM(units = 25, return_sequences = True))
    regressor.add(Dropout(0.2))

    # Adding a third LSTM layer and some Dropout regularisation
    regressor.add(LSTM(units = 25, return_sequences = True))
    regressor.add(Dropout(0.2))

    # Adding a fourth LSTM layer and some Dropout regularisation
    regressor.add(LSTM(units = 25))
    regressor.add(Dropout(0.2))

    # Adding the output layer
    regressor.add(Dense(units = num_emotions))
    
    return regressor

In [351]:
# Compiling the RNN
regressor.compile(optimizer = 'adam', metrics=['accuracy'], loss = 'mean_squared_error')
#Fitting to training set
regressor.fit(X_train, y_train, epochs = 25)



Epoch 1/25
25974/25974 [==============================] - 17s 637us/step - loss: 0.0240 - acc: 0.9157
Epoch 2/25
25974/25974 [==============================] - 12s 470us/step - loss: 0.0236 - acc: 0.9177
Epoch 3/25
25974/25974 [==============================] - 12s 462us/step - loss: 0.0239 - acc: 0.9175
Epoch 4/25
25974/25974 [==============================] - 12s 470us/step - loss: 0.0238 - acc: 0.9171
Epoch 5/25
25974/25974 [==============================] - 12s 468us/step - loss: 0.0233 - acc: 0.9191
Epoch 6/25
25974/25974 [==============================] - 12s 467us/step - loss: 0.0237 - acc: 0.9173
Epoch 7/25
25974/25974 [==============================] - 12s 472us/step - loss: 0.0232 - acc: 0.9202
Epoch 8/25
25974/25974 [==============================] - 12s 468us/step - loss: 0.0232 - acc: 0.9205
Epoch 9/25
25974/25974 [==============================] - 12s 473us/step - loss: 0.0232 - acc: 0.9193
Epoch 10/25
25974/25974 [==============================] - 12s 469us/step - loss: 

In [587]:
regressor.evaluate(X_test, y_test)

6494/6494 [==============================] - 2s 297us/step


[0.05276456107346735, 0.7685555896466792]

In [65]:
def prediction(text, model):
    vec_text=transform_t6_input(text)
    pred=model.predict(vec_text)
    pred=np.mean(pred, axis=0)
    return pred

In [90]:
def input_duplicator(text):
    X=pd.Series(text).apply(clean)
    splits=word_splits(X)
    numbers_series=splits
    num_docs=len(numbers_series)
    for index, doc in enumerate(numbers_series):
        print(len(doc))
        while len(doc)<7:
            orig_doc=doc.copy()
            orig_doc=list(orig_doc)
            doc=list(doc)
            for word in orig_doc:
                doc.append(word)
                #doc=np.insert(doc,(len(doc)),word, axis=0)
                #doc=np.append(doc, word, axis=1)
            modified=True
        numbers_series.iloc[index]=np.array(doc)
    X_1 = []
    if num_docs>1:
        for index in range(0, num_docs):
            doc=numbers_series.iloc[index]
            for i in range(6, len(doc)):
                X_1.append(doc[i-6:i])
    else:
        doc=numbers_series.iloc[0]
        print(doc.shape)
        for i in range(6, len(doc)):
                X_1.append(doc[i-6:i])
    return X_1

In [569]:
input_duplicator('I love everybody')

2
(8,)


[array(['love', 'everybody', 'love', 'everybody', 'love', 'everybody'],
       dtype='<U9'),
 array(['everybody', 'love', 'everybody', 'love', 'everybody', 'love'],
       dtype='<U9')]

In [571]:
prediction('love')

1
(8, 300)


array([[ 6.0217679e-03,  6.1481744e-03,  1.6123056e-03, -7.1944296e-04,
         2.5430262e-02,  9.4908869e-01,  7.6540709e-03],
       [ 6.0217679e-03,  6.1481744e-03,  1.6123056e-03, -7.1944296e-04,
         2.5430262e-02,  9.4908869e-01,  7.6540709e-03]], dtype=float32)

<h3>Importing amazon reviews sample</h3>

In [572]:
reviews=pd.read_csv("amazon_reviews_utf.csv")

In [581]:
reviews['Review Text'].iloc[1].split('.')[0]

'I bought this umbrella a few months ago and unfortunately, it broke while it was windy out'

In [578]:
reviews

,Name,Star Rating,Verifed Review,Helpful Vote Count,Date,Author,Author Link,Review Link,Review Text
0,"So far, so good (and dry)",5,YES,25,"November 9, 2016",S. R. Southard,https://amazon.com//gp/profile/amzn1.account.A...,https://amazon.com//gp/customer-reviews/R3V8GI...,I had my first opportunity to use the umbrella...
1,Not really a lifetime warranty...,2,YES,On,"July 10, 2018",Rico,https://amazon.com//gp/profile/amzn1.account.A...,https://amazon.com//gp/customer-reviews/R1ZA3B...,I bought this umbrella a few months ago and un...
2,"""THIS IS THE ONLY UMBRELLA YOU SHOULD EVER PUR...",5,YES,On,"January 22, 2018",On The Fly,https://amazon.com//gp/profile/amzn1.account.A...,https://amazon.com//gp/customer-reviews/RG5V0S...,First of all: I have no financial interest in ...
3,Most good quality umbrellas can't stand up to ...,3,YES,2,"April 25, 2018",Linann,https://amazon.com//gp/profile/amzn1.account.A...,https://amazon.com//gp/customer-reviews/R2EYVB...,"I work in New Haven CT, and in the fall, and w..."
4,Decent umbrella compromised by poor-quality ri...,3,YES,3,"October 15, 2017",Edward Ripley-Duggan,https://amazon.com//gp/profile/amzn1.account.A...,https://amazon.com//gp/customer-reviews/R1L9TU...,The umbrella appears to be robust; I used it r...
5,it is still like new. Great construction,4,YES,On,"August 14, 2016",MC Oddslice,https://amazon.com//gp/profile/amzn1.account.A...,https://amazon.com//gp/customer-reviews/R2W29X...,"Update: Too small! If there is wind, you will ..."
6,For a tiny person,2,YES,On,"May 25, 2018",AR,https://amazon.com//gp/profile/amzn1.account.A...,https://amazon.com//gp/customer-reviews/R3FPDT...,I was excited to try out this umbrella based o...
7,"Decent, a bit pricey?",4,YES,2,"September 2, 2017",Alex Wang,https://amazon.com//gp/profile/amzn1.account.A...,https://amazon.com//gp/customer-reviews/R12Q4A...,This umbrella is pretty good. never had a umbr...
8,Not consistent with reviews or description.,2,YES,3,"June 20, 2017",Hannah,https://amazon.com//gp/profile/amzn1.account.A...,https://amazon.com//gp/customer-reviews/R1UX1K...,I must have received a bad item or something b...
9,Seems well-built and well-designed.,5,YES,0,"January 26, 2018",Terende,https://amazon.com//gp/profile/amzn1.account.A...,https://amazon.com//gp/customer-reviews/R3C9IS...,I bought two of these for my and my spouse's u...


In [580]:
prediction(reviews['Review Text'].iloc[1].split('.')[0])

7
(7, 300)


array([0.05101114, 0.04537321, 0.715312  , 0.04185817, 0.03482933,
       0.05562435, 0.05663225], dtype=float32)

<h3>Importing DepecheMood</h3>

In [582]:
depeche=pd.read_csv('depeche_word_isolation.csv')

In [584]:
depeche.columns

Index(['Lemma#PoS', 'AFRAID', 'AMUSED', 'ANGRY', 'ANNOYED', 'DONT_CARE',
       'HAPPY', 'INSPIRED', 'SAD'],
      dtype='object')

In [ ]:
anger='ANGRY'+'ANNOYED'
disgust='ANGRY'
fear='AFRAID'
guilt='SAD'


<h3>Importing fb-reactions</h3>

In [94]:
fb_reactions=pd.read_json('data/fb_reactions.json')

In [95]:
fb_reactions['avg_wow']=fb_reactions['fb_wow']/fb_reactions['fb_like']

In [96]:
fb_reactions['message'].replace('', np.nan, inplace=True)

In [97]:
fb_reactions.dropna(subset=['message'], inplace=True)

In [98]:
fb_reactions=fb_reactions.sort_values('avg_wow', ascending=False)

In [99]:
fb_reactions=fb_reactions.reindex()

In [100]:
fb_reactions['emotion']='surprise'

In [101]:
fb_reactions.columns

Index(['database', 'date_only', 'day_name', 'description', 'external_picture',
       'fanpagelink', 'fb_angry', 'fb_haha', 'fb_id', 'fb_like', 'fb_love',
       'fb_sad', 'fb_thankful', 'fb_total_reactions', 'fb_wow',
       'highest_reaction', 'highest_reaction_extended', 'id', 'link',
       'message', 'name', 'num_comments', 'page_id', 'preprocessed_name',
       'preprocessed_stem_stop', 'shares', 'time_created', 'time_only', 'type',
       'avg_wow', 'emotion'],
      dtype='object')

In [102]:
surprise=fb_reactions.iloc[:1080]
surprise_X=surprise['message']
surprise_y=surprise['emotion']

In [103]:
surprise_df=pd.DataFrame()
surprise_df[0]=surprise_y
surprise_df[1]=surprise_X

In [104]:
surprise_df

,0,1
27823,surprise,Terrifying.
27382,surprise,This is terrifying.
28303,surprise,Yikes!
33513,surprise,For some reason this is a little hard to belie...
49643,surprise,Passengers aboard Delta flight 762 experienced...
28081,surprise,The 33-year-old officer managed to avoid serio...
33611,surprise,DEVELOPING...
27464,surprise,An industrial-sized barbecue grill broke free ...
7140,surprise,Oh my dear god.
27269,surprise,Awful.


In [105]:
isear[0].replace(['shame', 'disgust', 'guilt'], np.nan, inplace=True)
isear.dropna(subset=[0], inplace=True)

In [106]:
indico_emotions=isear.append(surprise_df)

In [107]:
indico_emotions

,0,1
0,joy,On days when I feel close to my partner and ot...
1,fear,Every time I imagine that someone I love or I ...
2,anger,When I had been obviously unjustly treated and...
3,sadness,When I think about the short time that we live...
7,joy,After my girlfriend had taken her exam we went...
8,fear,"When, for the first time I realized the meanin..."
9,anger,When a car is overtaking another and I am forc...
10,sadness,When I recently thought about the hard work it...
14,joy,When I pass an examination which I did not thi...
15,fear,When one has arranged to meet someone and that...


<h3>Indico validation</h3>

In [108]:
import indicoio
indicoio.config.api_key='397e82142cfdfe487c61dca634280b49'

In [111]:
indico_emotions[1].iloc[-1]

"Naturally, they're marketing this water as Purple Rain."

In [112]:
test_pred_1=indicoio.emotion(indico_emotions[1].iloc[-1])

In [114]:
test_pred_1['surprise']

0.23341253399848938

In [117]:
indico_emotions[1].iloc[:3]

0    On days when I feel close to my partner and ot...
1    Every time I imagine that someone I love or I ...
2    When I had been obviously unjustly treated and...
Name: 1, dtype: object

In [118]:
test_pred_mult=indicoio.emotion(list(indico_emotions[1].iloc[:3]))

In [119]:
test_pred_mult

[{'anger': 0.11947168409824371,
  'fear': 0.29070523381233215,
  'joy': 0.22359329462051392,
  'sadness': 0.33788686990737915,
  'surprise': 0.02834296226501465},
 {'anger': 0.3306489884853363,
  'fear': 0.10204938054084778,
  'joy': 0.035173580050468445,
  'sadness': 0.5207858085632324,
  'surprise': 0.011342236772179604},
 {'anger': 0.271008163690567,
  'fear': 0.07341042160987854,
  'joy': 0.03585120290517807,
  'sadness': 0.5347471237182617,
  'surprise': 0.08498305082321167}]

In [124]:
surprise_dict={}
for index, pred in enumerate(test_pred_mult):
    print(pred['surprise'], index)
    surprise_dict[index]=pred['surprise']
surprise_dict

0.02834296226501465 0
0.011342236772179604 1
0.08498305082321167 2


{0: 0.02834296226501465, 1: 0.011342236772179604, 2: 0.08498305082321167}

In [126]:
import operator
sorted_surprise_dict=sorted(surprise_dict.items(), key=operator.itemgetter(1))
sorted_surprise_dict

[(1, 0.011342236772179604), (0, 0.02834296226501465), (2, 0.08498305082321167)]

In [125]:
for index in surprise_dict:
    print(indico_emotions[1].iloc[index])

On days when I feel close to my partner and other friends.   
When I feel at peace with myself and also experience a close  
contact with people whom I regard greatly.
Every time I imagine that someone I love or I could contact a  
serious illness, even death.
When I had been obviously unjustly treated and had no possibility  
of elucidating this.


<h3>Evaluating many facebook entries in the message column</h3>

In [127]:
surprise=fb_reactions.iloc[:4000]
surprise_X=surprise['message']
len(surprise_X)

4000

<h1>Do NOT run this next step again, it counts against the quota</h1>

#Indico_surprise_predictions=indicoio.emotion(list(surprise_X))

In [136]:
import pickle
pickle.dump(Indico_surprise_predictions, open('Indico_surprise_predictions.pkl', 'wb'))

In [140]:
def sort_surprise(indico_list):
    surprise_dict={}
    for index, pred in enumerate(indico_list):
        surprise_dict[index]=pred['surprise']
    sorted_surprise_dict=sorted(surprise_dict.items(), key=operator.itemgetter(1), reverse=True)
    return sorted_surprise_dict

In [141]:
sorted_surprise=sort_surprise(Indico_surprise_predictions)

In [142]:
sorted_surprise

[(3683, 0.9539512395858765),
 (3883, 0.9527742862701416),
 (829, 0.949898898601532),
 (447, 0.9370132684707642),
 (2890, 0.9296550750732422),
 (2145, 0.9246888756752014),
 (3447, 0.9224801063537598),
 (2809, 0.9215435981750488),
 (2163, 0.9202695488929749),
 (151, 0.9162212610244751),
 (1989, 0.9157223701477051),
 (3298, 0.9139634966850281),
 (1070, 0.901566743850708),
 (800, 0.897817850112915),
 (749, 0.8887519836425781),
 (1754, 0.887529730796814),
 (3439, 0.8798812031745911),
 (548, 0.8695153594017029),
 (1268, 0.8664361238479614),
 (2858, 0.8638198375701904),
 (1512, 0.8535745143890381),
 (3850, 0.8436046838760376),
 (861, 0.8278483152389526),
 (1348, 0.8221002817153931),
 (1773, 0.8197835683822632),
 (2017, 0.8197430372238159),
 (1732, 0.8158684968948364),
 (3125, 0.811977744102478),
 (1006, 0.8094233274459839),
 (394, 0.808263897895813),
 (1647, 0.8046984076499939),
 (2420, 0.8036953210830688),
 (3146, 0.7918825149536133),
 (103, 0.7886497378349304),
 (2585, 0.7864409685134888),


In [148]:
high_surprise={}
for key, value in sorted_surprise:
    if value>0.5:
        high_surprise[key]=value
len(high_surprise)

154

In [147]:
fb_reactions.iloc[3872]['message']

'Think of these lesser-known places before you book your next vacation.'

In [144]:
fb_reactions.iloc[3255]['message']

"Once you cook salmon like this, you'll never want it any other way."

<h3>Importing Semeval</h3>

In [150]:
semeval_emotions=pd.read_csv('data/semeval_emotions.csv')

In [165]:
semeval_emotions.rename({'0': 0, '1': 1}, axis='columns', inplace=True)

In [166]:
isear_plus_semeval=isear.append(semeval_emotions)

In [225]:
isear_plus_semeval

,0,1
0,joy,On days when I feel close to my partner and ot...
1,fear,Every time I imagine that someone I love or I ...
2,anger,When I had been obviously unjustly treated and...
3,sadness,When I think about the short time that we live...
7,joy,After my girlfriend had taken her exam we went...
8,fear,"When, for the first time I realized the meanin..."
9,anger,When a car is overtaking another and I am forc...
10,sadness,When I recently thought about the hard work it...
14,joy,When I pass an examination which I did not thi...
15,fear,When one has arranged to meet someone and that...


In [219]:
def input_duplicator_train(text):
    splits=text.split(' ')
    while len(splits)<7:
        orig_doc=splits.copy()
        for word in orig_doc:
            splits.append(word)
    return ' '.join(splits)

In [226]:
X=isear_plus_semeval[1].apply(clean).apply(input_duplicator_train)
y=isear_plus_semeval[0]

In [261]:
X

0       day feel close partner friend feel peace also ...
1       every time imagine someone love could contact ...
2       obviously unjustly treated possibility elucida...
3       think short time live relate period life think...
7       girlfriend taken exam went parent place girlfr...
8       when first time realized meaning death when fi...
9       car overtaking another forced drive road car o...
10      recently thought hard work take study one want...
14      pas examination think well pas examination thi...
15      one arranged meet someone person arrives late ...
16      one unjustly accused something one done one un...
17      one study seem hopelessly difficult uninterest...
21        passing exam expect pas passing exam expect pas
22      climbed tree pick apple angle ladder enable ge...
24        child child child child child child child child
25      2 year old son climbed sat 7th floor balcony l...
26      partner attacked lost three teeth partner atta...
27            

In [267]:
not_count=0
not_docs=[]
for row in X:
    split=row.split(' ')
    for word in split:
        if word=='not':
            not_count+=1
            not_docs.append(' '.join(split))
not_count
not_docs

['receiving postcard form sweetheart abroad vacation sure would write not',
 'friend promised would definitely call tell visiting not',
 'person died not really friend person died not really friend',
 'person died not really friend person died not really friend',
 'encountered father lack understanding also unwillingness understand parent far thing not he prof right concerned',
 'anger men inability human not men anger men inability human not men',
 'anger men inability human not men anger men inability human not men',
 'sexually assaulted not rape passionate kissing friend father stopped driving home wanted lift refused 3 time felt itd rude refuse again really afraid might do',
 'argument best friend thought right not argument best friend thought right not',
 'argument best friend thought right not argument best friend thought right not',
 'bought new bed apt squeak move it tried fix could not',
 'get admission msc bsc felt really bad started journey bombay accepted uncle offer come b

In [228]:
X_1, y_1=transform_t6_train(X)

In [230]:
encoder=LabelEncoder()
encoder.fit(y_1)
y_1=encoder.transform(y_1)
y_1=np_utils.to_categorical(y_1)
y_1

array([[0., 0., 1., 0.],
       [0., 0., 1., 0.],
       [0., 0., 1., 0.],
       ...,
       [0., 0., 0., 1.],
       [0., 0., 0., 1.],
       [0., 0., 0., 1.]], dtype=float32)

In [231]:
X_train, X_test, y_train, y_test = train_test_split(X_1, y_1, test_size = 0.2, random_state = 0)
X_train, y_train = np.array(X_train), np.array(y_train)

In [238]:
four_emotions=create_lstm(4)

C:\Users\Nicholas\AppData\Local\Programs\Python\Python36\lib\site-packages\ipykernel_launcher.py:6: UserWarning: The `input_dim` and `input_length` arguments in recurrent layers are deprecated. Use `input_shape` instead.
  
C:\Users\Nicholas\AppData\Local\Programs\Python\Python36\lib\site-packages\ipykernel_launcher.py:6: UserWarning: Update your `LSTM` call to the Keras 2 API: `LSTM(units=25, return_sequences=True, input_shape=(None, 300...)`
  


In [258]:
four_emotions.compile(optimizer = 'adam', metrics=['accuracy'], loss = 'mean_squared_error')
four_emotions.fit(X_train, y_train, epochs = 25)

Epoch 1/25
27013/27013 [==============================] - 16s 596us/step - loss: 0.0415 - acc: 0.9079
Epoch 2/25
27013/27013 [==============================] - 13s 465us/step - loss: 0.0402 - acc: 0.9114
Epoch 3/25
27013/27013 [==============================] - 13s 468us/step - loss: 0.0394 - acc: 0.9124
Epoch 4/25
27013/27013 [==============================] - 13s 467us/step - loss: 0.0379 - acc: 0.9176
Epoch 5/25
27013/27013 [==============================] - 13s 466us/step - loss: 0.0371 - acc: 0.9194
Epoch 6/25
27013/27013 [==============================] - 13s 468us/step - loss: 0.0373 - acc: 0.9202
Epoch 7/25
27013/27013 [==============================] - 14s 511us/step - loss: 0.0354 - acc: 0.9242
Epoch 8/25
27013/27013 [==============================] - 15s 544us/step - loss: 0.0346 - acc: 0.9269
Epoch 9/25
27013/27013 [==============================] - 15s 565us/step - loss: 0.0348 - acc: 0.9266
Epoch 10/25
27013/27013 [==============================] - 14s 526us/step - loss: 

In [243]:
X_test, y_test= np.array(X_test), np.array(y_test)

In [259]:
four_emotions.evaluate(X_test, y_test) 

6754/6754 [==============================] - 1s 215us/step


[0.05646390239088301, 0.8606751554634291]

In [268]:
four_emotions.save('four_emotions.h5')

In [275]:
i_eat=prediction('I eat', four_emotions)

1
(8, 300)


<h3> Synsets from wordnet </h3>

In [481]:
synonyms=[]

for syn in wordnet.synsets('joy', pos=wordnet.NOUN):
    for l in syn.lemmas():
        synonyms.append(l.name())
synonyms=set(synonyms)
synonyms

{'delight', 'joy', 'joyfulness', 'joyousness', 'pleasure'}

In [344]:
def syn_finder(syns, part_of_speech):
    new_syns=list(syns)
    if len(new_syns)<100:
        for syn in syns:
            for synonym in wordnet.synsets(syn, pos=part_of_speech):
                for l in synonym.lemmas():
                    new_syns.append(l.name())
        new_syns=list(set(new_syns))
        return syn_finder(new_syns, part_of_speech)
    else:
        return new_syns

In [345]:
test=syn_finder(['joy'], wordnet.NOUN)

In [346]:
test

['amour',
 'quality',
 'chromosome_mapping',
 'drug_abuse',
 'engagement',
 'betrothal',
 'treatment',
 'thing',
 'workout',
 'fiber',
 'fibre',
 'eccentric',
 'joyfulness',
 'study',
 'white_plague',
 'use',
 'subroutine',
 'section',
 'conclusion',
 'pattern',
 'portion',
 'spending',
 'enjoyment',
 'mesh',
 'physical_exercise',
 'type',
 'case',
 'position',
 'purport',
 'component_part',
 'aim',
 'ingestion',
 'joy',
 'booking',
 'single-valued_function',
 'use_of_goods_and_services',
 'office',
 'model',
 'region',
 'expending',
 'inlet',
 'act',
 'conflict',
 'share',
 'occasion',
 'wont',
 'function',
 'object',
 'phthisis',
 'bearing',
 'deterrent_example',
 'manipulation',
 'affaire',
 'object_lesson',
 'breathing_in',
 'excogitation',
 'utilisation',
 'juncture',
 'usage',
 'government_agency',
 'finding',
 'Mandrillus_leucophaeus',
 'joyousness',
 'inspiration',
 'intention',
 'post',
 'habit',
 'social_function',
 'decision',
 'recital',
 'good_example',
 'practice_session'

<h3>DepecheMood

In [4]:
depeche=pd.read_csv('data/depeche_word_isolation.csv')

In [449]:
depeche

,Lemma#PoS,AFRAID,AMUSED,ANGRY,ANNOYED,DONT_CARE,HAPPY,INSPIRED,SAD
0,0,0.087697,0.092489,0.154124,0.077188,0.136462,0.287111,0.084152,0.080778
1,1,0.129418,0.108454,0.114674,0.114568,0.136147,0.161294,0.114729,0.120718
2,10,0.130558,0.109739,0.115716,0.109559,0.151769,0.140957,0.111359,0.130342
3,100,0.156990,0.126365,0.102937,0.087751,0.121395,0.109795,0.118774,0.175993
4,1000,0.133133,0.186631,0.096675,0.060371,0.051486,0.151350,0.120990,0.199363
5,10000,0.079639,0.114055,0.016001,0.005296,0.643330,0.050498,0.016048,0.075132
6,1000th,0.000000,0.291220,0.000000,0.000000,0.000000,0.437545,0.271235,0.000000
7,100th,0.308124,0.115666,0.100621,0.102881,0.037436,0.118386,0.150157,0.066728
8,101st,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000
9,10th,0.092306,0.066443,0.122701,0.086613,0.109534,0.181413,0.101312,0.239679


In [453]:
depeche[(depeche['HAPPY']>.5) & (depeche['HAPPY']<1.0)]

,Lemma#PoS,AFRAID,AMUSED,ANGRY,ANNOYED,DONT_CARE,HAPPY,INSPIRED,SAD
12,115th,0.095784,0.043246,0.080320,0.033227,0.082486,0.549403,0.040278,0.075256
101,55th,0.000000,0.000000,0.000000,0.000000,0.000000,0.546478,0.234474,0.219049
128,a-line,0.000000,0.000000,0.480277,0.000000,0.000000,0.519723,0.000000,0.000000
182,abila,0.000000,0.067542,0.018817,0.083031,0.038649,0.505564,0.286397,0.000000
243,absinthe,0.000000,0.053181,0.000000,0.040860,0.000000,0.732600,0.173359,0.000000
249,absolutist,0.021433,0.101608,0.000000,0.029740,0.000000,0.788576,0.042446,0.016196
266,abstruse,0.000000,0.000000,0.112527,0.031033,0.308162,0.513133,0.000000,0.035144
269,absurdly,0.000000,0.000000,0.330582,0.000000,0.000000,0.669418,0.000000,0.000000
282,aby,0.000000,0.000000,0.000000,0.000000,0.000000,0.584599,0.000000,0.415401
287,acacia,0.000000,0.185767,0.000000,0.043269,0.057065,0.604080,0.109819,0.000000


In [456]:
depeche_freq=pd.read_csv("data/DepecheMood_V1.0/DepecheMood_freq.txt", sep='\t')
depeche_freq

,Lemma#PoS,AFRAID,AMUSED,ANGRY,ANNOYED,DONT_CARE,HAPPY,INSPIRED,SAD
0,0#n,0.091040,0.101889,0.167116,0.065456,0.135221,0.291064,0.064975,0.083239
1,1#n,0.121637,0.116173,0.118003,0.119131,0.138837,0.164305,0.105911,0.116004
2,10#n,0.131860,0.118659,0.121393,0.111911,0.138485,0.143611,0.106803,0.127277
3,100#n,0.170863,0.128193,0.104132,0.093903,0.107457,0.117811,0.106983,0.170658
4,1000#n,0.147027,0.142151,0.076382,0.065647,0.060772,0.155713,0.094969,0.257339
5,10000#n,0.094633,0.172659,0.011131,0.003737,0.586478,0.046655,0.009551,0.075156
6,1000th#a,0.000000,0.336853,0.000000,0.000000,0.000000,0.413456,0.249691,0.000000
7,100th#a,0.320738,0.146574,0.118702,0.081067,0.027685,0.114831,0.120825,0.069577
8,101st#a,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000
9,10th#a,0.096738,0.064845,0.122668,0.099643,0.123646,0.186206,0.097122,0.209132


In [461]:
depeche_freq[((depeche_freq['HAPPY']>.75) & (depeche_freq['HAPPY']<1.0)) | ((depeche_freq['INSPIRED']>.75) & (depeche_freq['INSPIRED']<1.0))]

,Lemma#PoS,AFRAID,AMUSED,ANGRY,ANNOYED,DONT_CARE,HAPPY,INSPIRED,SAD
243,absinthe#n,0.00000,0.055164,0.000000,0.040001,0.000000,0.761721,0.143115,0.000000
365,accrued#a,0.00000,0.000000,0.000000,0.000000,0.000000,0.807161,0.192839,0.000000
411,acoustician#n,0.00000,0.000000,0.000000,0.000000,0.000000,0.826408,0.173592,0.000000
525,adhd#n,0.00000,0.000000,0.000000,0.000000,0.000000,0.182276,0.817724,0.000000
533,adios#n,0.00000,0.000000,0.121665,0.000000,0.000000,0.043214,0.835120,0.000000
595,adorned#a,0.00000,0.094897,0.000000,0.068813,0.000000,0.759185,0.035171,0.041934
951,alca#n,0.00000,0.000000,0.000000,0.000000,0.000000,0.121257,0.878743,0.000000
960,alcove#n,0.00000,0.000000,0.000000,0.000000,0.000000,0.093786,0.906214,0.000000
1029,allegorical#a,0.00000,0.000000,0.000000,0.000000,0.000000,0.093786,0.906214,0.000000
1129,altruistic#a,0.00000,0.000000,0.000000,0.000000,0.000000,0.197831,0.802169,0.000000


<i>Getting 'happy' words programmatically has proved haphazard. I will use an API and then manually remove words that don't fit.

<h3>Getting joy words</h3>

In [5]:
compare_words=['joy', 'joyful', 'happy', 'happiness', 'excitement', 'amusement', 'bliss', 'charm', 'cheer', 'comfort', 'delight', 'elation', 'glee', 'humor', 'satisfaction', 'wonder']
adjectives=[]
nouns=[]
verbs=[]
word_types=[adjectives, nouns, verbs]
for word in compare_words:
    r=requests.get('http://words.bighugelabs.com/api/2/050617425e370b0aa06ac00be6ed9cc9/'+word+'/json')
    js=r.json()
    for index, word_type in enumerate(['adjective', 'noun', 'verb']):
        if word_type in js:
            if 'rel' in js[word_type]:
                for word in js[word_type]['rel']:
                    word_types[index].append(word)
            if 'sim' in js[word_type]:
                for word in js[word_type]['sim']:
                    word_types[index].append(word)
            if 'syn' in js[word_type]:
                for word in js[word_type]['syn']:
                    word_types[index].append(word)
adjectives=list(set(adjectives))
nouns=list(set(nouns))
verbs=list(set(verbs))
bad_nouns=['damages', 'body substance', 'redress', 'spoken language', 'subject matter', 'liquid body substance', 'inflammation', 'speech communication', 'psychological state', 'substance', 'happening', 'voice communication', 'condition', 'physical object', 'positive stimulus']
better_nouns=[noun for noun in nouns if noun not in bad_nouns]
joy_words=adjectives+better_nouns+verbs
len(joy_words)

216

<h3>Getting sadness words

In [6]:
compare_words=['sad', 'sadness', 'dreary', 'bitter', 'sorrow', 'wistful', 'heartbroken', 'melancholy', 'mournful', 'pessimistic', 'somber', 'dismal']
adjectives=[]
nouns=[]
verbs=[]
word_types=[adjectives, nouns, verbs]
for word in compare_words:
    r=requests.get('http://words.bighugelabs.com/api/2/050617425e370b0aa06ac00be6ed9cc9/'+word+'/json')
    js=r.json()
    for index, word_type in enumerate(['adjective', 'noun', 'verb']):
        if word_type in js:
            if 'rel' in js[word_type]:
                for word in js[word_type]['rel']:
                    word_types[index].append(word)
            if 'sim' in js[word_type]:
                for word in js[word_type]['sim']:
                    word_types[index].append(word)
            if 'syn' in js[word_type]:
                for word in js[word_type]['syn']:
                    word_types[index].append(word)
adjectives=list(set(adjectives))
nouns=list(set(nouns))
verbs=list(set(verbs))
bad_nouns=['negative stimulus', 'feeling', 'gustatory perception', 'taste perception', 'taste property' ,'gustatory sensation', 'humour', 'liquid body substance', 'taste sensation', 'bodily fluid', 'body fluid']
better_nouns=[noun for noun in nouns if noun not in bad_nouns]
if 'change taste' in verbs:
    verbs.remove('change taste')
sad_words=adjectives+better_nouns+verbs
len(sad_words)

92

<h3>Getting anger words

In [7]:
compare_words=['anger', 'angry', 'mad', 'fury', 'furious', 'hatred', 'annoyance', 'annoying', 'outrage', 'irritation', 'ire', 'rage']
adjectives=[]
nouns=[]
verbs=[]
word_types=[adjectives, nouns, verbs]
for word in compare_words:
    r=requests.get('http://words.bighugelabs.com/api/2/050617425e370b0aa06ac00be6ed9cc9/'+word+'/json')
    js=r.json()
    for index, word_type in enumerate(['adjective', 'noun', 'verb']):
        if word_type in js:
            if 'rel' in js[word_type]:
                for word in js[word_type]['rel']:
                    word_types[index].append(word)
            if 'sim' in js[word_type]:
                for word in js[word_type]['sim']:
                    word_types[index].append(word)
            if 'syn' in js[word_type]:
                for word in js[word_type]['syn']:
                    word_types[index].append(word)
adjectives=list(set(adjectives))
nouns=list(set(nouns))
verbs=list(set(verbs))
bad_nouns=['negative stimulus', 'fashion', 'manic disorder', 'hysteria', 'intensity' ,'fierceness', 'craze', 'emotion', 'fad', 'abnormalcy', 'mythical creature']
better_nouns=[noun for noun in nouns if noun not in bad_nouns]
angry_words=adjectives+better_nouns+verbs
len(angry_words)

176

<h3>Getting fearful words

In [8]:
compare_words=['fear', 'fearful', 'fright', 'frightened', 'afraid', 'despair', 'dread', 'horrified', 'panic', 'terror', 'terrified', 'worry', 'unease', 'uneasiness']
adjectives=[]
nouns=[]
verbs=[]
word_types=[adjectives, nouns, verbs]
for word in compare_words:
    r=requests.get('http://words.bighugelabs.com/api/2/050617425e370b0aa06ac00be6ed9cc9/'+word+'/json')
    js=r.json()
    for index, word_type in enumerate(['adjective', 'noun', 'verb']):
        if word_type in js:
            if 'rel' in js[word_type]:
                for word in js[word_type]['rel']:
                    word_types[index].append(word)
            if 'sim' in js[word_type]:
                for word in js[word_type]['sim']:
                    word_types[index].append(word)
            if 'syn' in js[word_type]:
                for word in js[word_type]['syn']:
                    word_types[index].append(word)
adjectives=list(set(adjectives))
nouns=list(set(nouns))
verbs=list(set(verbs))
bad_nouns=['negative stimulus', 'someone', 'feeling', 'status', 'veneration' ,'individual', 'monkey', 'brat', 'emotion', 'person', 'awe']
better_nouns=[noun for noun in nouns if noun not in bad_nouns]
fear_words=adjectives+better_nouns+verbs
len(fear_words)

168

In [28]:
negative

['not',
 'no',
 'neither',
 'nor',
 'but',
 'however',
 'although',
 'nonetheless',
 'despite',
 'except',
 'even though',
 'yet']

In [9]:
emotions={'anger':angry_words, 'fear':fear_words, 'joy':joy_words, 'sadness':sad_words}
negative_words=['not', 'no', 'neither', 'nor']
emotion_opp={'anger':'joy', 'fear':'joy', 'joy':'sadness', 'sadness':'joy'}
negative_X=[]
negative_y=[]
for emotion in emotions:
    for neg_word in negative_words:
        for emot_word in emotions[emotion]:
            negative_X.append(neg_word+' '+emot_word+' '+neg_word+' '+emot_word+' '+neg_word+' '+emot_word+' '+neg_word+' '+emot_word)
            negative_y.append(emotion_opp[emotion])

In [28]:
neg_X=pd.Series(negative_X)
neg_y=pd.Series(negative_y)
series=[neg_y,neg_X]
neg_df=pd.concat(series, axis=1)
neg_df

,0,1
0,joy,not hot under the collar not hot under the col...
1,joy,not irascible not irascible not irascible not ...
2,joy,not mad not mad not mad not mad
3,joy,not furious not furious not furious not furious
4,joy,not angry not angry not angry not angry
5,joy,not excited not excited not excited not excited
6,joy,not irritating not irritating not irritating n...
7,joy,not wrathful not wrathful not wrathful not wra...
8,joy,not smouldering not smouldering not smoulderin...
9,joy,not umbrageous not umbrageous not umbrageous n...


In [20]:
neg_df[1].value_counts()

joy        1744
sadness     864
Name: 1, dtype: int64

In [29]:
neg_df.to_csv('data/constructed_negations.csv', header=-1, index=False)

In [30]:
neg_df_test=pd.read_csv('data/constructed_negations.csv')

<h3>Old code

In [383]:
not_X=[]
not_y=[]
no_X=[]
no_y=[]

lists=[not_X, not_y, no_X, no_y]
words={'not': [not_X, not_y], 'no': [no_X, no_y]}
for syn in test:
    for word in words:
        words[word][0].append(word+' '+syn+' '+word+' '+syn+' '+word+' '+syn+' '+word+' '+syn)
        words[word][1].append('sadness')
not_X=pd.Series(not_X)
not_y=pd.Series(not_y)
no_X=pd.Series(no_X)
no_y=pd.Series(no_y)
negation_X=not_X.append(no_X)
negation_y=not_y.append(no_y)

In [43]:
def transform_t6_negation_0(text):
    X=pd.Series(text).apply(clean)
    splits=word_splits(X)
    numbers_series=splits.apply(vec_word)
    num_docs=len(numbers_series)
    X_1 = []
    for index in range(0, num_docs):
        doc=numbers_series.iloc[index]
        for i in range(6, len(doc)):
            X_1.append(doc[i-6:i])
    return X_1

In [56]:
def transform_t6_negation(text):
    X=pd.Series(text).apply(clean)
    splits=word_splits(X)
    numbers_series=splits.apply(vec_word)
    num_docs=len(numbers_series)
    X_1 = []
    y_1 = []
    for index in range(0, num_docs):
        doc=numbers_series.iloc[index]
        for i in range(6, len(doc)):
            X_1.append(doc[i-6:i])
            y_1.append(negative_y1[index])
    return X_1, y_1

In [38]:
joy=np.array([0,0,1,0])

In [39]:
sad=np.array([0,0,0,1])

In [40]:
negative_y1=[]
for emotion in negative_y:
    if emotion=='joy':
        negative_y1.append(joy)
    else:
        negative_y1.append(sad)
negative_y1

[array([0, 0, 1, 0]),
 array([0, 0, 1, 0]),
 array([0, 0, 1, 0]),
 array([0, 0, 1, 0]),
 array([0, 0, 1, 0]),
 array([0, 0, 1, 0]),
 array([0, 0, 1, 0]),
 array([0, 0, 1, 0]),
 array([0, 0, 1, 0]),
 array([0, 0, 1, 0]),
 array([0, 0, 1, 0]),
 array([0, 0, 1, 0]),
 array([0, 0, 1, 0]),
 array([0, 0, 1, 0]),
 array([0, 0, 1, 0]),
 array([0, 0, 1, 0]),
 array([0, 0, 1, 0]),
 array([0, 0, 1, 0]),
 array([0, 0, 1, 0]),
 array([0, 0, 1, 0]),
 array([0, 0, 1, 0]),
 array([0, 0, 1, 0]),
 array([0, 0, 1, 0]),
 array([0, 0, 1, 0]),
 array([0, 0, 1, 0]),
 array([0, 0, 1, 0]),
 array([0, 0, 1, 0]),
 array([0, 0, 1, 0]),
 array([0, 0, 1, 0]),
 array([0, 0, 1, 0]),
 array([0, 0, 1, 0]),
 array([0, 0, 1, 0]),
 array([0, 0, 1, 0]),
 array([0, 0, 1, 0]),
 array([0, 0, 1, 0]),
 array([0, 0, 1, 0]),
 array([0, 0, 1, 0]),
 array([0, 0, 1, 0]),
 array([0, 0, 1, 0]),
 array([0, 0, 1, 0]),
 array([0, 0, 1, 0]),
 array([0, 0, 1, 0]),
 array([0, 0, 1, 0]),
 array([0, 0, 1, 0]),
 array([0, 0, 1, 0]),
 array([0,

In [57]:
negative_X1, negative_y1=np.array(transform_t6_negation(negative_X))

In [420]:
negated_X=transform_t6_negation_0(negation_X)
negated_X=np.array(negated_X)

In [418]:
sads=[]
for _ in range(len(negated_X)):
    sads.append(sad)
negated_y=np.array(sads)

In [421]:
four_emotions.fit(negated_X, negated_y, epochs = 25)

Epoch 1/25
620/620 [==============================] - 1s 1ms/step - loss: 0.1254 - acc: 0.7016
Epoch 2/25
620/620 [==============================] - 0s 523us/step - loss: 0.0091 - acc: 0.9823
Epoch 3/25
620/620 [==============================] - 0s 521us/step - loss: 0.0060 - acc: 0.9903
Epoch 4/25
620/620 [==============================] - 0s 508us/step - loss: 0.0050 - acc: 0.9935
Epoch 5/25
620/620 [==============================] - 0s 511us/step - loss: 0.0048 - acc: 0.9935
Epoch 6/25
620/620 [==============================] - 0s 511us/step - loss: 0.0042 - acc: 0.9935
Epoch 7/25
620/620 [==============================] - 0s 505us/step - loss: 0.0041 - acc: 0.9935
Epoch 8/25
620/620 [==============================] - 0s 458us/step - loss: 0.0038 - acc: 0.9935
Epoch 9/25
620/620 [==============================] - 0s 463us/step - loss: 0.0037 - acc: 0.9935
Epoch 10/25
620/620 [==============================] - 0s 453us/step - loss: 0.0038 - acc: 0.9935
Epoch 11/25
620/620 [==========

In [426]:
prediction('I am not not happy', four_emotions)

array([-3.8905169e-03,  1.7428547e-03, -2.0447250e-04,  9.9890691e-01],
      dtype=float32)

In [66]:
four_emotions.fit(negative_X1, negative_y1, epochs=25)

ValueError: Error when checking input: expected lstm_5_input to have 3 dimensions, but got array with shape (5688, 1)

<h3>Further options</h3>

I could 

1.Gather and combine many emotional labelled datasets to have larger training data

2.Create new training data synthetically

3.Create labelled data using word averages (auto-labelling) data

4.Use Indico to get around 10,000 more labelled training data rows

<strong>Idea:</strong>Duplicate input if length of input too short

<strong>Idea:</strong>Branch out to several models depending on input size

<strong>Indico's emotions</strong>:
Anger, Fear, Joy, Sadness, Surprise

<strong>ISEAR's emotions</strong>: Anger, Fear, Joy, Sadness, Disgust, Shame, Guilt


Four emotions:
Anger, Fear, Joy, Sadness

Disgust goes with Anger, Surprise goes with Fear

<h3>Plan for API</h3>

1.For now, retrain model with just four emotions. Use ISEAR and Semeval for better accuracy. Redeploy with good formatting

2.For the future, 
Download facebook data tagged with 'Wow' emoji predominantly for 'surprise' (This results in a low yield of actually surprised texts, about 150/ every 4000)
To use Indico to find the surprising texts, in order to get to around 1000 surprising texts it will cost around $150. If we don't use multiple fake accounts.

Alternative option: Get Facebook scraper approved in order to download facebook comments from "wow" stories (might have a higher yield of surprised texts)

Continue looking for source of texts labelled with surprise (couldn't find yet)

Redo four_emotions model with categorical cross entropy loss to get probabilities

<h1>Input shape

3D tensor with shape (batch_size, timesteps, input_dim)</h1>

0:anger

1:disgust

2:fear

3:guilt

4: joy

5:sadness

6:shame

[anger, fear, joy, sadness]

In [315]:
def prediction_json(text, model):
    vec_text = transform_t6_input(text)
    pred = model.predict(vec_text)
    pred = np.mean(pred, axis=0)
    anger=pred[0]
    fear=pred[1]
    joy=pred[2]
    sadness=pred[3]
    response_dict={'anger': anger, 'fear': fear, 'joy': joy, 'sadness': sadness, 'surprise': 0}
    return jsonify(response_dict)

In [318]:
def prediction_str(text, model):
    vec_text = transform_t6_input(text)
    pred = model.predict(vec_text)
    pred = np.mean(pred, axis=0)
    anger=pred[0]
    fear=pred[1]
    joy=pred[2]
    sadness=pred[3]
    response_dict = {'anger': anger, 'fear': fear, 'joy': joy, 'sadness': sadness, 'surprise': 0}
    #return jsonify(response_dict)
    return str(response_dict)

In [319]:
prediction_str('I like pie', four_emotions)

2
(8, 300)


"{'anger': 0.31061664, 'fear': 0.037538864, 'joy': 0.5914509, 'sadness': 0.06033083, 'surprise': 0}"